# Načítání dat ze souboru

## Čistý python

In [ ]:
with open("pyvenv.cfg", "r") as f:
    data = f.read()

for line in data.splitlines():
    print(line)

In [ ]:
with open("pyvenv.cfg", "r") as f:
    data = f.readlines()

settings = {}
for line in data:
    line = line.strip()
    key, val = line.split(" = ") # tuple/list unpacking
    settings[key] = val
    
with open("pyvenv.dict", "w") as f:
    for key, val in settings.items():
        f.write('settings["{}"] = "{}"\n'.format(key, val))

del(settings)
settings = {}
with open("pyvenv.dict") as f:
    lines = f.readlines()
    for line in lines:
        exec(line)
print(settings)

In [ ]:
f = open("pyvenv.cfg", "r")

print(f.readline().strip())
print(f.readline().strip())
f.close()
f.closed

## csv

In [ ]:
with open("topeni.txt", "r") as f:
    print(f.readline())
    print(f.readline())
    
with open("topeni.txt", "r") as f:
    data = []
    line = f.readline()
    while line:
        line = line.strip()
        if line[0] in ["#", "!"]:
            line = f.readline()
            continue
        data.append(line.split(" "))
        line = f.readline()
        
mesice = [line[1] for line in data[:12]]
teplo = [float(line[2]) for line in data[:12]]

import matplotlib.pyplot as plt
plt.plot(mesice, teplo)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data = np.loadtxt("topeni.txt").transpose()
plt.plot(data[1][:12], data[2][:12])
plt.show()

In [ ]:
years = np.unique(data[0].astype("int"))

yearly_data = np.array_split(np.delete(data, 0, 0).transpose(), len(years))
yearly_data = [x.transpose() for x in yearly_data]

for i, year in enumerate(years):
    plt.plot(yearly_data[i][0], yearly_data[i][1], label = year)
plt.legend()
plt.show()

https://docs.python.org/3/library/csv.html#csv.reader

In [ ]:
import csv
with open("topeni.txt") as csvfile:
    reader = csv.reader(csvfile, delimiter = ' ')
    for row in reader:
        print(row)

## JSON
https://docs.python.org/3/library/json.html

In [ ]:
import json
with open('example.json', 'r') as data:
    bib = json.load(data)

def dict_pretty_print(d, indent = 0, indchar = "  "):
    for key, val in d.items():
        if type(val) == dict:
            print(indchar * indent + "{}:".format(key))
            dict_pretty_print(val, indent = indent + 1, indchar = indchar)
        else:
            print(indchar * indent + "{}: {}".format(key, val))

dict_pretty_print(bib)

In [ ]:
import json
with open('bibliography.json', 'r') as data:
    bib = json.load(data)

dict_pretty_print(bib[0])

In [ ]:
def get_author_string(entry):
    authors = []
    append_etall = False
    for name in entry["author"]:
        try:
            authors.append("{}, {}.".format(name["family"], name["given"][0]))
        except:
            continue
    if len(authors) > 3:
        authors = authors[:4]
        append_etall = True
        
    authorstring = ", ".join(authors)
    if append_etall:
        authorstring += " et al."
    return authorstring
    
def print_bib_entry_nicely(entry):
    authorstring = get_author_string(entry)
    print("{0}: {1}".format(authorstring, entry["title"]))
    
for entry in bib:
    print_bib_entry_nicely(entry)

## XML - eXtensible Markup Language

https://docs.python.org/3/library/xml.etree.elementtree.html

In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse('sample.xml')
root = tree.getroot()

for child in root:
    for grandchild in child:
        print(grandchild.text)
        
for neighbor in root.iter('neighbor'):
    print(neighbor.attrib)

## .ini
https://docs.python.org/3/library/configparser.html

In [ ]:
import configparser as cp
config = cp.ConfigParser()
config.read("sample.ini")

for section in config:
    print(section)
    for entry in config[section]:
        print("|- {}: {}".format(entry, config[section][entry]))

## Excel
https://xlrd.readthedocs.io/

## priprava dat

In [ ]:
import pandas as pd
temps = pd.read_csv("data/teploty_short.dat", header = None, delimiter = " ")
temps = temps.rename(columns={0 : "timestamp", 1 : "temperature"})
temps.set_index(['timestamp'], drop = True, inplace = True)
temps.index = pd.to_datetime(temps.index)
temps2 = temps.resample('1D').mean()
temps.plot()
temps2.plot()

In [ ]:
temps3 = temps.resample('15Min').mean()
groups = []
for index, group in temps3.groupby(temps3.index.date):
    groups.append(group)
    
with open("data.html", "w") as f:
    f.write("<html>\n")
    f.write("<head><title>Temperature data</title></head><link rel=\"stylesheet\" href=\"style.css\"><body><div id=\"content\" class=\"content\">\n")
    for group in groups:
        f.write("<table>\n")
        f.write("<caption>{}</caption>\n".format(group.index[0].date()))
        for date, row in group.iterrows():
            f.write("<tr><td>{}</td><td>{:.1f}</td></tr>\n".format(date.time(), row['temperature']))
        f.write("</table>\n")
    f.write("</div></body></html>\n")

## Webscraping - beautiful soup

instalace
```bash
pip3 install bs4 lxml html5lib
```

In [ ]:
from bs4 import BeautifulSoup

with open("data/sample.html") as fp:
    soup = BeautifulSoup(fp, 'html.parser')

a = soup.table
print(a.name)
print(a.attrs)

In [ ]:
for child in a.children:
    print(child)

print(a.caption)

In [ ]:
for td in soup.find_all('td'):
    print(td.string)

In [ ]:
soup.find(id="sample_table")

In [ ]:
import requests
# requests - jednoduchy http interface
# pip3 install requests

r = requests.get("http://vaclav-alt.github.io/data/index.html")
soup = BeautifulSoup(r.content)
print(soup.title)

for table in soup.find_all("table"):
    print(table.caption)

In [ ]:
import pandas as pd
tbls = soup.find_all("table")

rows = tbls[0].find_all("tr")

dates = []
temps = []

for row in rows:
    cols = row.find_all("td")
    dates.append(cols[0].text)
    temps.append(cols[1].text)
    
df = pd.DataFrame({
    "date" : dates,
    "temps" : temps
})
df = df.astype({"temps" : float})
df.plot()

## HTTP, API

https://mixedanalytics.com/blog/list-actually-free-open-no-auth-needed-apis/

In [ ]:
import requests
from IPython.display import Image, display

url = "https://dog.ceo/api/breeds/image/random"
r = requests.get(url)
if r.status_code == 200:
    data = r.json()
    image_url = data["message"]
    display(Image(image_url))

https://github.com/15Dkatz/official_joke_api

https://sv443.net/jokeapi/v2/
```json
{
    "error": false,
    "category": "Programming",
    "type": "single",
    "joke": "Java is like Alzheimer's, it starts off slow, but eventually, your memory is gone.",
    "flags": {
        "nsfw": false,
        "religious": false,
        "political": false,
        "racist": false,
        "sexist": false
    },
    "id": 27,
    "lang": "en"
}
```

In [ ]:
import json

# url = "https://official-joke-api.appspot.com/random_ten"
url = "https://sv443.net/jokeapi/v2/joke/Any"


r = requests.get(url)
if r.status_code == 200:
    data = json.loads(r.text)
    print(json.dumps(data, indent = 2))

In [ ]:
import time
url = "https://sv443.net/jokeapi/v2/joke/Programming/rand"
url = "https://sv443.net/jokeapi/v2/joke/Any?blacklistFlags=nsfw,religious,political,racist,sexist"

r = requests.get(url)
if r.status_code == 200:
    data = json.loads(r.text)
    if data["type"] == "single":
        print(data["joke"])
    else:
        print(data["setup"])
        time.sleep(3)
        print(data["delivery"])

### UNICORN

In [ ]:
def dict_pretty_print(d, indent = 0, indchar = "  "):
    for key, val in d.items():
        if type(val) == dict:
            print(indchar * indent + "{}:".format(key))
            dict_pretty_print(val, indent = indent + 1, indchar = indchar)
        else:
            print(indchar * indent + "{}:\t{}".format(key, val))

In [ ]:
url1 = "https://uuappg01-eu-w-1.plus4u.net/ucl-weatherstation-maing01/ecdf51f870ae48b69e10fcd6cc2c7ef3/weatherConditions/getLast?code=PETROVKA&type=1"
url2 = "https://uuappg01-eu-w-1.plus4u.net/ucl-weatherstation-maing01/590f7ec72ad34126a5650fdfd807b3b6/weatherConditions/getLast?code=STRACHOVICE&type=1"
url3 = "https://uuappg01-eu-w-1.plus4u.net/ucl-weatherstation-maing01/36edc26c55a345798f3642ba8d22d80b/weatherConditions/getLast?code=MELTEMI&type=1"

import requests
import json
r = requests.get(url1)

if r.status_code == 200:
    data = json.loads(r.text)

dict_pretty_print(data)